In [1]:
#folderpath = r'/Users/rebeccakrall/Downloads';

folderpath = r'C:\Users\natet\Desktop\Experimental_Data\Control\c_129'
animal = 'c_129';

In [2]:
from data_cleaning import mat_struct_2_pd, clean_behavior_data
b,t = mat_struct_2_pd(folderpath, animal)
b2 = clean_behavior_data(b)

In [3]:
def create_model_inputs(b2):
    
    import numpy as np
    
    # Eventually, try to make this smarter/adaptable. RN, just pick covariates
    d = b2.iloc[:, [1,3,4,5,8]].copy()
    d['bias'] = np.ones(len(d))
    X = d.to_numpy()
    x_names = list(d.columns)
    
    y = b2['Rightward'].to_numpy()
    session = np.array([b2.index[b2['Session']== s][0] for s in np.unique(b2['Session'])])
    
    return X, y, session
    
    

In [4]:
x,y,session = create_model_inputs(b2)


In [5]:
N = x.shape[0] # number of data/time points for the indirect pathway cohort
K = 3 # number of latent states
C = 2 # number of observation classes
D = x.shape[1] # number of GLM inputs (regressors)

In [6]:

from glmhmm import glm, glm_hmm
from glmhmm.utils import permute_states, find_best_fit, compare_top_weights
from glmhmm.visualize import plot_model_params, plot_loglikelihoods, plot_weights
import numpy as np

import matplotlib.pyplot as plt
import numpy as np
import time
from sklearn.model_selection import KFold


import matplotlib as mpl

In [7]:

real_GLMHMM = glm_hmm.GLMHMM(N,D,C,K,observations="bernoulli",gaussianPrior=1)

In [8]:
inits = 20 # set the number of initializations
maxiter = 250 # maximum number of iterations of EM to allow for each fit
tol = 1e-3

# store values for each initialization
lls_all = np.zeros((inits,250))
A_all = np.zeros((inits,K,K))
w_all = np.zeros((inits,K,D,C))

# fit the model for each initialization
print('fits for indirect pathway cohort')
for i in range(inits):
    t0 = time.time()
    # initialize the weights
    A_init,w_init,pi_init = real_GLMHMM.generate_params(weights=['GLM',-0.2,1.2,x,y,1])
    # fit the model                     
    lls_all[i,:],A_all[i,:,:],w_all[i,:,:],pi0 = real_GLMHMM.fit(y,x,A_init,w_init, maxiter=maxiter,tol=tol,sess=session) 
    minutes = (time.time() - t0)/60
    print('initialization %s complete in %.2f minutes' %(i+1, minutes))

fits for indirect pathway cohort


c:\ProgramData\Anaconda3\lib\site-packages\scipy\optimize\_differentiable_functions.py:141: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  fx = np.asarray(fx).item()


ValueError: The user-provided objective function must return a scalar value.

In [9]:
import numpy as np
x = np.load(r'C:\Users\natet\Downloads\indirect_x.npy')
y = np.load(r'C:\Users\natet\Downloads\indirect_y.npy')
session = np.load(r'C:\Users\natet\Downloads\indirect_sessions.npy')

In [40]:
session.shape

(272,)

In [26]:
session.shape

(19,)